### 最近在项目中有很多用到dataframe处理搜集的数据场景，这里记录一些使用dataframe的总结

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from IPython.display import display

### dataframe的创建

In [2]:
# 直接使用DataFrame 创建
pd.DataFrame([[1,1], [2,2], [3,3], [4,4]], columns=list('AB'))
# 从csv文件中读取
pd.read_csv('./cluster.csv')

# 从ndarray构建
pd.DataFrame(np.random.randn(3,4), columns=list('ABCD'))

# 从Series中构建
d = pd.Series([1,2,3,4,5,6], dtype='float')
pd.DataFrame(d)

# 从Series中可以创建一些有意思的类型，比如日期等
#d = pd.date_range('20130101', periods=6)
d = pd.Series(['2013-01-01', '2013-01-02', '2013-01-03'], \
          dtype='datetime64[ns]')
pd.DataFrame(np.random.randn(3,3), index=d)

# 分别构建dataframe中的每一列，来构建最终的dataframe
# DataFrame中的每一列应该是一个series(todo)
df1 = pd.DataFrame({ 'A' : 1.,
                    'B' : pd.Timestamp('20130102'),
                    'C' : pd.Series(1,index=list(range(4)),dtype='float32')})
display (df1)

,A,B,C
0,1.0,2013-01-02,1.0
1,1.0,2013-01-02,1.0
2,1.0,2013-01-02,1.0
3,1.0,2013-01-02,1.0


### dataframe过滤，筛选满足条件的记录

In [3]:
ft = pd.read_csv('./cluster.csv')
display (ft.head(5))

,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen,cluster
0,3,12669,9656,7561,214,2674,1338,1
1,3,7057,9810,9568,1762,3293,1776,1
2,3,6353,8808,7684,2405,3516,7844,1
3,3,13265,1196,4221,6404,507,1788,0
4,3,22615,5410,7198,3915,1777,5185,1


In [74]:
display (ft[(ft['cluster'] == 1) & (ft['Fresh'] > 10000)]. head(5))
# (ft['cluster'] == 1).shape print (type(ft['cluster'] == 1))
# 输出由bool值构成的Series 然后dataframe根据这个Series来做过滤。

,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen,cluster
0,3,12669,9656,7561,214,2674,1338,1
4,3,22615,5410,7198,3915,1777,5185,1
6,3,12126,3199,6975,480,3140,545,1
12,3,31714,12319,11757,287,3881,2931,1
13,3,21217,6208,14982,3095,6707,602,1


In [75]:
t = pd.DataFrame(ft['Fresh'].apply(lambda x: str(x)))
t[t['Fresh'].str.contains('1266')]
# 我们需要按照条件输出一个bool数组来告诉DataFrame哪些记录需要保存下来
# 使用哪些方法可以由我们自己来选择。

,Fresh
0,12669


In [7]:
### 使用Group BY 来进行分类统计

In [8]:

dfg = pd.DataFrame({'col_1': pd.Series([i for i in range(1, 7)]),
                    'col_2': pd.Series([j*10 for j in range(1, 7)]),
              'type' : pd.Series(['t1','t1','t1',
                                't2','t2','t2'])})
#print (dfg.index)

In [9]:
# 统计每种type的个数
dfgb = dfg.groupby(['type', 'col_2']).count()
print (dfgb.index)
display (dfgb)

MultiIndex(levels=[['t1', 't2'], [10, 20, 30, 40, 50, 60]],
           labels=[[0, 0, 0, 1, 1, 1], [0, 1, 2, 3, 4, 5]],
           names=['type', 'col_2'])


col_1
type col_2       
t1   10         1
     20         1
     30         1
t2   40         1
     50         1
     60         1

### 多表之间的链接操作

In [10]:
# 使用 concat缺省值会被自动不上NaN
df1 = pd.DataFrame([[1,2,3], [4,5,6], [1,2,3]], columns=list('ABC'))
df2 = pd.DataFrame([[7,8,9], [10,11,12], [1,4,4]], columns=list("ABD"))

display(pd.concat([df1,df2]))

,A,B,C,D
0,1,2,3.0,NaN
1,4,5,6.0,NaN
2,1,2,3.0,NaN
0,7,8,NaN,9.0
1,10,11,NaN,12.0
2,1,4,NaN,4.0


### 使用left inner right 分别做表连接
- inner链接相当于去两个表,满足条件的交集，不满足条件的行则会被去掉。
- left 链接除了取两个表的交集，还会保留left表中没有匹配上的行记录。
- right 和left相反会保留right表中没有匹配上的行记录

In [11]:
display(pd.merge(left=df1, right=df2, on= ['A'], how='inner'))

,A,B_x,C,B_y,D
0,1,2,3,4,4
1,1,2,3,4,4


In [12]:
display(pd.merge(left=df1, right=df2, on= ['A'], how='left'))

,A,B_x,C,B_y,D
0,1,2,3,4.0,4.0
1,4,5,6,NaN,NaN
2,1,2,3,4.0,4.0


In [13]:
display(pd.merge(left=df1, right=df2, on= ['A'], how='right'))

,A,B_x,C,B_y,D
0,1,2.0,3.0,4,4
1,1,2.0,3.0,4,4
2,7,NaN,NaN,8,9
3,10,NaN,NaN,11,12


### DataFrame 中的时间类型操作
- 按照频率来进行统计，可以使用dataframe的resample函数 

In [14]:
one_day = ['2017/10/1 12:52:01', '2017/10/1 13:52:01', '2017/10/1 14:52:01']
one_week = ['2017/10/1 12:52:01', '2017/10/2 13:52:01', '2017/10/3 14:52:01']
one_day.extend(one_week)
income = [100, 200, 300, 400, 1, 2]
df_time = pd.DataFrame({'datatime': one_day,
              'income': income})
df_time['datatime'] = df_time['datatime'].apply(lambda x : dt.datetime.strptime(x, '%Y/%m/%d %H:%M:%S'))
df_time = df_time.set_index('datatime')
# 按照天来统计收入情况
df_time.resample('1d').sum()

,income
datatime,
2017-10-01,1000
2017-10-02,1
2017-10-03,2


### dataframe中的axis参数相关
- dataframe中有很多操作需要区分是对行还是列进行操作，比如sum,mean()这样的统计操作，还有一些删除操作也需要指名是对行还是列进行操作。

In [15]:
df_drop = pd.DataFrame([[1,1,1,1], [2,2,2,2],[3,3,3,3]], columns=list('ABCD'))
display (df_drop)

,A,B,C,D
0,1,1,1,1
1,2,2,2,2
2,3,3,3,3


In [16]:
# 当 axis = 0 时代表的是对行进行求和，把每一行给加起来，所以最终记录个数为
display (df_drop.sum(axis=0).shape)
# axis = 1 时代表是对列进行求和
display (df_drop.sum(axis=1).shape)

(4,)

(3,)

In [17]:
# axis=0 按照行来进行数据删除操作
display (df_drop.drop(0, axis=0).shape)
# axis=1 按照列来进行数据删除操作
display (df_drop.drop('A', axis=1).shape)

(2, 4)

(3, 3)

### DataFrame的转置
- 思路是先把数据矩阵转置，然后在把index 和 col重新赋值。

In [58]:
df = pd.DataFrame([[1,1,1], [2,2,2], [3,3,3], [4,4,4]], columns=list('ABC'))
df_t = pd.DataFrame(df.values.copy().T, columns=df.index.tolist())
df_t.index = df.columns.tolist()
display (df)
display (df_t)

,A,B,C
0,1,1,1
1,2,2,2
2,3,3,3
3,4,4,4


,0,1,2,3
A,1,2,3,4
B,1,2,3,4
C,1,2,3,4


### 在DataFrame中添加删除 行和列
- 添加不存在的行和列
- 修改已经存在的行和列
- 修改指定的行和列

In [87]:
# 在DataFrame 有很多种取出行和列的方法，loc,iloc等。其中loc里面需要填入的是index值，iloc类似于数组的下标索引(从0开始)。
# DataFrame 的下标操作同样支持切片操作。
df = pd.DataFrame(columns=['col_1', 'col_2', 'col_3'])
for i in range(1, 6):
    # 如果索引i在Dataframe中不存在则新行的添加
    df.loc[i] = 0

# 这里的2是可以取到的
df.loc[1:2] = 1
df['col_4'] = 4

# loc 和 iloc 不仅仅可以对行进行操作，还可以对列进行操作，访问的方法和二维数组差不多
df.loc[1,'col_2':'col_3'] = 3
# 这里的切片操作因为是下标索引，所以取不到2
df.iloc[1, 0:2] = 2
display (df)

,col_1,col_2,col_3,col_4
1,1,3,3,4
2,2,2,1,4
3,0,0,0,4
4,0,0,0,4
5,0,0,0,4
